In [44]:
import pandas as pd
import numpy as np
import geopandas as gpd
import transbigdata as tbd
import os
import glob 
from shapely.wkt import loads
import warnings
warnings.filterwarnings('ignore')

In [45]:
charge_grid_1 = pd.read_csv('data/订单量0712.csv')
charge_grid_2 = pd.read_csv('data/订单量0715.csv')
charge_grid = pd.concat([charge_grid_1,charge_grid_2])
charge_grid.drop_duplicates(subset=['经纬度编号'],inplace=True)
# 生成栅格范围
def generate_grid(grid_heatmap):
    grid_heatmap['lon'] = grid_heatmap['经纬度编号'].str.split('-').str[0].astype(float)
    grid_heatmap['lat'] = grid_heatmap['经纬度编号'].str.split('-').str[1].astype(float)
    params = {'slon': 120,
    'slat': 30,
    'deltalon': 0.01,
    'deltalat': 0.01,
    'theta': 0,
    'method': 'rect',
    'gridsize': 500}
    grid_heatmap['LONCOL'],grid_heatmap['LATCOL'] = tbd.GPS_to_grid(grid_heatmap['lon'], grid_heatmap['lat'],params)
    grid_heatmap['geometry'] = tbd.grid_to_polygon([grid_heatmap['LONCOL'], grid_heatmap['LATCOL']],params)
    grid_heatmap = gpd.GeoDataFrame(grid_heatmap, geometry='geometry')
    return grid_heatmap
Charge_Grid = generate_grid(charge_grid)

In [46]:
gdf_charge_grid = gpd.GeoDataFrame(
    charge_grid,
    geometry='geometry'
)
gdf_charge_grid

,日期,充电时段,经纬度编号,充电次数,总充电量,BEV车辆数,PHEV车辆数,lon,lat,LONCOL,LATCOL,geometry
0,2023-07-12,0-1,121.46-31.34,100,"3,046.6",96,4,121.46,31.34,146,134,"POLYGON ((121.45500 31.33500, 121.46500 31.335..."
1,2023-07-12,22-23,121.37-31.32,100,"2,144.8",58,40,121.37,31.32,137,132,"POLYGON ((121.36500 31.31500, 121.37500 31.315..."
2,2023-07-12,22-23,121.57-31.16,98,"2,253.42",54,43,121.57,31.16,157,116,"POLYGON ((121.56500 31.15500, 121.57500 31.155..."
3,2023-07-12,22-23,121.39-31.15,97,"2,603.53",67,30,121.39,31.15,139,115,"POLYGON ((121.38500 31.14500, 121.39500 31.145..."
4,2023-07-12,22-23,121.4-31.29,85,"1,848.78",49,36,121.40,31.29,140,129,"POLYGON ((121.39500 31.28500, 121.40500 31.285..."
...,...,...,...,...,...,...,...,...,...,...,...,...
4534,2023-07-12,8-9,121.64-31.29,1,6.48,0,1,121.64,31.29,164,129,"POLYGON ((121.63500 31.28500, 121.64500 31.285..."
4555,2023-07-12,8-9,121.36-31.31,1,2.41,0,1,121.36,31.31,136,131,"POLYGON ((121.35500 31.30500, 121.36500 31.305..."
4558,2023-07-12,8-9,121.48-31.13,1,0.74,0,1,121.48,31.13,148,113,"POLYGON ((121.47500 31.12500, 121.48500 31.125..."
4568,2023-07-12,8-9,121.47-31.14,1,12.92,0,1,121.47,31.14,147,114,"POLYGON ((121.46500 31.13500, 121.47500 31.135..."


In [47]:
gdf_charge_grid.crs = {'init':'epsg:4326'}

In [48]:
## 计算上海市的面积
shanghai_area = pd.read_csv('./data/shanghai.csv')
## 转换成polygon数据
shanghai_area['geometry'] = shanghai_area['geometry'].apply(lambda x: loads(x) if isinstance(x, object) else None)
## dataframe转换成geodataframe
shanghai_area = gpd.GeoDataFrame(shanghai_area,geometry='geometry')
shanghai_area.drop(columns=['index_right'],inplace=True)
## 计算建筑物面积
def building_area(gdf):
    gdf.crs = {'init':'epsg:4326'}
    ## 使用适配于上海的投影坐标系
    tmp = gdf.to_crs(epsg=4528)
    gdf['area'] = tmp.area
    ## 计算总面积
    gdf['area'] = gdf['area'] * gdf['height']
    return gdf
shanghai_area = building_area(shanghai_area)

In [49]:
gdf_charge_grid['index'] = [i for i in range(len(gdf_charge_grid))]

In [50]:
station_sjoined_area = gpd.sjoin(gdf_charge_grid,shanghai_area)
station_sjoined_area.drop(columns=['index_right','building_id'],inplace=True)
station_sjoined_area = station_sjoined_area.groupby(['index','geometry','LONCOL','LATCOL','lon','lat'])['area'].sum().reset_index()
station_sjoined_area = gpd.GeoDataFrame(station_sjoined_area,
                                        geometry='geometry')

In [51]:
def read_poi(folder_path):
    csv_file_pattern = os.path.join(folder_path, '*.csv')  # 构建文件匹配模式
    csv_files = glob.glob(csv_file_pattern)  # 获取所有.csv文件的路径列表
    data = pd.DataFrame()
    for file_path in csv_files:
        df = pd.read_csv(file_path,on_bad_lines='skip')
        ## 处理tsv文件
        if len(df.columns) == 1:
            df = pd.read_csv(file_path,sep='\t')
        data = pd.concat([data,df])
    return data

In [52]:
poi1 = read_poi('./POI')
poi2 = read_poi('./POI2')
poi3 = read_poi('./POI长三角')
poi1 = poi1[poi1['province']=='上海市']
poi2 = poi2[poi2['province']=='上海市']
poi3 = poi3[poi3['province']=='上海市']
## 合并上海市所有的POI信息
poi_shanghai = pd.concat([poi1,poi2,poi3])
poi_shanghai.drop_duplicates(subset=['uid'],inplace=True)
poi_shanghai = gpd.GeoDataFrame(poi_shanghai,
                                geometry=gpd.points_from_xy(poi_shanghai['lon'],poi_shanghai['lat']))


In [53]:
tech_park_poi = poi_shanghai[poi_shanghai['name'].str.contains('科技园')]
tech_park_poi['technology_park'] = 1
tech_park_poi = tech_park_poi[['technology_park','geometry']]
car_park_poi = poi_shanghai[poi_shanghai['name'].str.contains('停车场')]
car_park_poi['car_park'] = 1
car_park_poi = car_park_poi[['car_park','geometry']]
supermarket_poi = poi_shanghai[(poi_shanghai['name'].str.contains('超市'))&(poi_shanghai['tag'].str.contains('超市'))]
supermarket_poi['supermarket'] = 1
supermarket_poi = supermarket_poi[['supermarket','geometry']]
tag_not_nan_poi = poi_shanghai.dropna(subset=['tag'])
residential_poi = tag_not_nan_poi[tag_not_nan_poi['tag'].str.contains('住宅区')]
residential_poi['residential'] = 1
residential_poi = residential_poi[['residential','geometry']]
commercial_poi = poi_shanghai[poi_shanghai['type']=='shopping']
commercial_poi['commercial'] = 1
commercial_poi = commercial_poi[['commercial','geometry']]

In [54]:
def merge_info_to_charge_station(charge_station,merge_poi,column):
    tmp_sjoined = gpd.sjoin(charge_station,merge_poi)
    tmp_sjoined[column] = tmp_sjoined[column].fillna(0)
    myid_agg = tmp_sjoined.groupby(['index'])[column].sum().reset_index()
    return myid_agg

In [55]:
technology_agg = merge_info_to_charge_station(station_sjoined_area,tech_park_poi,'technology_park')

In [56]:
car_agg = merge_info_to_charge_station(station_sjoined_area,car_park_poi,'car_park')
supermarket_agg = merge_info_to_charge_station(station_sjoined_area,supermarket_poi,'supermarket')
residential_agg = merge_info_to_charge_station(station_sjoined_area,residential_poi,'residential')
commercial_agg = merge_info_to_charge_station(station_sjoined_area,commercial_poi,'commercial')

In [57]:
geographic_info = pd.merge(technology_agg,car_agg,on=['index'],how='outer')

In [58]:

geographic_info = pd.merge(geographic_info,supermarket_agg,on=['index'],how='outer')
geographic_info = pd.merge(geographic_info,residential_agg,on=['index'],how='outer')
geographic_info = pd.merge(geographic_info,commercial_agg,on=['index'],how='outer')
## 合并地理信息
charge_station_feature = pd.merge(station_sjoined_area,geographic_info,on=['index'],how='left')


In [59]:
charge_station_feature.fillna(0,inplace=True)

In [60]:
charge_station_feature

,index,geometry,LONCOL,LATCOL,lon,lat,area,technology_park,car_park,supermarket,residential,commercial
0,0,"POLYGON ((121.45500 31.33500, 121.46500 31.335...",146,134,121.46,31.34,3.141321e+06,1.0,6.0,1.0,5.0,80.0
1,1,"POLYGON ((121.36500 31.31500, 121.37500 31.315...",137,132,121.37,31.32,3.792313e+06,0.0,2.0,8.0,20.0,109.0
2,2,"POLYGON ((121.56500 31.15500, 121.57500 31.155...",157,116,121.57,31.16,2.726178e+06,0.0,3.0,2.0,16.0,45.0
3,3,"POLYGON ((121.38500 31.14500, 121.39500 31.145...",139,115,121.39,31.15,3.883191e+06,0.0,10.0,3.0,13.0,59.0
4,4,"POLYGON ((121.39500 31.28500, 121.40500 31.285...",140,129,121.40,31.29,3.437720e+06,0.0,16.0,6.0,14.0,66.0
...,...,...,...,...,...,...,...,...,...,...,...,...
665,665,"POLYGON ((121.63500 31.28500, 121.64500 31.285...",164,129,121.64,31.29,4.276120e+05,0.0,0.0,1.0,3.0,5.0
666,666,"POLYGON ((121.35500 31.30500, 121.36500 31.305...",136,131,121.36,31.31,3.036253e+05,0.0,2.0,0.0,0.0,3.0
667,667,"POLYGON ((121.47500 31.12500, 121.48500 31.125...",148,113,121.48,31.13,8.439866e+05,0.0,1.0,1.0,0.0,2.0
668,668,"POLYGON ((121.46500 31.13500, 121.47500 31.135...",147,114,121.47,31.14,2.484398e+04,0.0,0.0,1.0,0.0,2.0


In [61]:
charge_copy = charge_station_feature.copy()

In [62]:
charge_copy.drop(columns=['index'],inplace=True)

In [63]:
charge_copy

,geometry,LONCOL,LATCOL,lon,lat,area,technology_park,car_park,supermarket,residential,commercial
0,"POLYGON ((121.45500 31.33500, 121.46500 31.335...",146,134,121.46,31.34,3.141321e+06,1.0,6.0,1.0,5.0,80.0
1,"POLYGON ((121.36500 31.31500, 121.37500 31.315...",137,132,121.37,31.32,3.792313e+06,0.0,2.0,8.0,20.0,109.0
2,"POLYGON ((121.56500 31.15500, 121.57500 31.155...",157,116,121.57,31.16,2.726178e+06,0.0,3.0,2.0,16.0,45.0
3,"POLYGON ((121.38500 31.14500, 121.39500 31.145...",139,115,121.39,31.15,3.883191e+06,0.0,10.0,3.0,13.0,59.0
4,"POLYGON ((121.39500 31.28500, 121.40500 31.285...",140,129,121.40,31.29,3.437720e+06,0.0,16.0,6.0,14.0,66.0
...,...,...,...,...,...,...,...,...,...,...,...
665,"POLYGON ((121.63500 31.28500, 121.64500 31.285...",164,129,121.64,31.29,4.276120e+05,0.0,0.0,1.0,3.0,5.0
666,"POLYGON ((121.35500 31.30500, 121.36500 31.305...",136,131,121.36,31.31,3.036253e+05,0.0,2.0,0.0,0.0,3.0
667,"POLYGON ((121.47500 31.12500, 121.48500 31.125...",148,113,121.48,31.13,8.439866e+05,0.0,1.0,1.0,0.0,2.0
668,"POLYGON ((121.46500 31.13500, 121.47500 31.135...",147,114,121.47,31.14,2.484398e+04,0.0,0.0,1.0,0.0,2.0


In [64]:
data = pd.read_csv('./output/charge_station_feature.csv')

In [65]:
grid_index = pd.read_csv('./output/grid_index.csv')

In [66]:
charge_copy = pd.merge(charge_copy,grid_index,on=['lon','lat'],how='inner')

In [67]:
charge_copy.to_csv('./output/grid_feature.csv')

In [68]:
data.columns, data['充电站所处道路属性'].unique()

(Index(['myid', '充电站名称', '充电站所处道路属性', 'lon', 'lat', '充电站所属区域分类', '充电站类型',
        '交流桩数量', '直流桩数量', '交流桩总装机功率', '直流桩总装机功率', 'geometry', 'station_sum',
        'area', 'technology_park', 'car_park', 'supermarket', 'residential',
        'commercial', 'index'],
       dtype='object'),
 array(['01城市内道路', '02城际间高速道路', nan], dtype=object))

In [69]:
data = data[['index','充电站所处道路属性','充电站所属区域分类','充电站类型','交流桩数量', '直流桩数量', '交流桩总装机功率', '直流桩总装机功率'
             ]]
data['充电站所处道路属性'].fillna('道路unknown',inplace=True)
data['充电站所属区域分类'].fillna('所属区域unknown',inplace=True)
data.fillna(0,inplace=True)

In [70]:
data['充电站所处道路属性'].unique(), data['充电站所属区域分类'].unique()

(array(['01城市内道路', '02城际间高速道路', '道路unknown'], dtype=object),
 array(['02居民区内部停车场', '01充电场站', '10大型建筑物配建停车场', '03公共机构内部停车场',
        '05写字楼内部停车场', '04企事业单位内部停车场', '08大型文体设施公共停车场', '09城市绿地公共停车场',
        '07交通枢纽公共停车场', '06工业园区内部停车场', '其它', '11路边停车位'], dtype=object))

In [71]:
# 定义你想要检查的列名
columns_to_check = ['充电站所处道路属性', '充电站所属区域分类', '充电站类型']

# 遍历每一列，并为每个可能的值创建新列
for col in columns_to_check:
    unique_values = data[col].unique()  # 获取所有可能的值
    for value in unique_values:
        # 为每个值创建一个新列，如果原始列中有这个值，则新列为1，否则为0
        data[value] = (data[col] == value).astype(int)

# 查看DataFrame的前几行以确认结果
data.head()

,index,充电站所处道路属性,充电站所属区域分类,充电站类型,交流桩数量,直流桩数量,交流桩总装机功率,直流桩总装机功率,01城市内道路,02城际间高速道路,...,09城市绿地公共停车场,07交通枢纽公共停车场,06工业园区内部停车场,其它,11路边停车位,01公共,05其他专用,06虚拟桩群,02公交车专用,03出租车专用
0,568,01城市内道路,02居民区内部停车场,01公共,12.0,0.0,42.0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1,568,01城市内道路,02居民区内部停车场,01公共,12.0,0.0,42.0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
2,349,01城市内道路,02居民区内部停车场,01公共,0.0,1.0,0.0,204,1,0,...,0,0,0,0,0,1,0,0,0,0
3,479,01城市内道路,01充电场站,01公共,0.0,5.0,0.0,400.01,1,0,...,0,0,0,0,0,1,0,0,0,0
4,479,01城市内道路,01充电场站,05其他专用,9.0,0.0,63.0,0,1,0,...,0,0,0,0,0,0,1,0,0,0


In [72]:
data['直流桩总装机功率'] = data['直流桩总装机功率'].astype(str)
data['直流桩总装机功率'] = data['直流桩总装机功率'].apply(lambda x: float(x.split('-')[0]))
data = data.groupby(['index']).agg({
    '交流桩数量':'sum',
    '直流桩数量':'sum',
    '交流桩总装机功率':'sum',
    '直流桩总装机功率':'sum',
    '01城市内道路':'sum',
    '02城际间高速道路':'sum',
    '道路unknown':'sum',
    '02居民区内部停车场':'sum',
    '01充电场站':'sum',  '10大型建筑物配建停车场':'sum',   '03公共机构内部停车场':'sum',    '05写字楼内部停车场':'sum',
        '04企事业单位内部停车场':'sum', '08大型文体设施公共停车场':'sum',   '09城市绿地公共停车场':'sum',   '07交通枢纽公共停车场':'sum',
         '06工业园区内部停车场':'sum', '其它':'sum', '11路边停车位':'sum', '01公共':'sum',
              '05其他专用':'sum',        '06虚拟桩群':'sum',       '02公交车专用':'sum',       '03出租车专用':'sum'
}).reset_index()

In [73]:
data

,index,交流桩数量,直流桩数量,交流桩总装机功率,直流桩总装机功率,01城市内道路,02城际间高速道路,道路unknown,02居民区内部停车场,01充电场站,...,09城市绿地公共停车场,07交通枢纽公共停车场,06工业园区内部停车场,其它,11路边停车位,01公共,05其他专用,06虚拟桩群,02公交车专用,03出租车专用
0,0,583.0,2.0,4081.0,120.0,16,0,0,4,10,...,0,0,0,0,0,2,1,12,1,0
1,1,40.0,60.0,262.5,3600.0,13,0,0,4,9,...,0,0,0,0,0,6,1,6,0,0
2,2,26.0,53.0,147.0,1985.0,19,0,0,6,11,...,0,0,0,0,0,6,1,12,0,0
3,3,80.0,2.0,413.0,80.0,24,0,0,8,10,...,0,0,0,0,0,10,4,10,0,0
4,4,58.0,0.0,449.0,0.0,11,0,0,3,8,...,0,0,0,0,0,1,1,9,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
639,665,198.0,7.0,1050.0,680.0,10,0,0,2,2,...,0,0,0,0,0,2,6,2,0,0
640,666,66.0,5.0,462.0,160.0,2,0,1,0,1,...,0,0,0,0,1,2,1,0,0,0
641,667,3.0,0.0,21.0,0.0,3,0,0,1,2,...,0,0,0,0,0,0,0,3,0,0
642,668,4.0,0.0,28.0,0.0,1,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0


In [74]:
charge_agg = pd.merge(charge_copy,data,on=['index'],how='inner')

In [75]:
charge_agg

,geometry,LONCOL,LATCOL,lon,lat,area,technology_park,car_park,supermarket,residential,...,09城市绿地公共停车场,07交通枢纽公共停车场,06工业园区内部停车场,其它,11路边停车位,01公共,05其他专用,06虚拟桩群,02公交车专用,03出租车专用
0,"POLYGON ((121.45500 31.33500, 121.46500 31.335...",146,134,121.46,31.34,3.141321e+06,1.0,6.0,1.0,5.0,...,0,0,0,0,0,2,1,12,1,0
1,"POLYGON ((121.36500 31.31500, 121.37500 31.315...",137,132,121.37,31.32,3.792313e+06,0.0,2.0,8.0,20.0,...,0,0,0,0,0,6,1,6,0,0
2,"POLYGON ((121.56500 31.15500, 121.57500 31.155...",157,116,121.57,31.16,2.726178e+06,0.0,3.0,2.0,16.0,...,0,0,0,0,0,6,1,12,0,0
3,"POLYGON ((121.38500 31.14500, 121.39500 31.145...",139,115,121.39,31.15,3.883191e+06,0.0,10.0,3.0,13.0,...,0,0,0,0,0,10,4,10,0,0
4,"POLYGON ((121.39500 31.28500, 121.40500 31.285...",140,129,121.40,31.29,3.437720e+06,0.0,16.0,6.0,14.0,...,0,0,0,0,0,1,1,9,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
639,"POLYGON ((121.63500 31.28500, 121.64500 31.285...",164,129,121.64,31.29,4.276120e+05,0.0,0.0,1.0,3.0,...,0,0,0,0,0,2,6,2,0,0
640,"POLYGON ((121.35500 31.30500, 121.36500 31.305...",136,131,121.36,31.31,3.036253e+05,0.0,2.0,0.0,0.0,...,0,0,0,0,1,2,1,0,0,0
641,"POLYGON ((121.47500 31.12500, 121.48500 31.125...",148,113,121.48,31.13,8.439866e+05,0.0,1.0,1.0,0.0,...,0,0,0,0,0,0,0,3,0,0
642,"POLYGON ((121.46500 31.13500, 121.47500 31.135...",147,114,121.47,31.14,2.484398e+04,0.0,0.0,1.0,0.0,...,0,0,0,0,0,1,0,0,0,0


In [76]:
charge_order_12 = pd.read_csv('./output/订单量0712.csv')
charge_order_15 = pd.read_csv('./output/订单量0715.csv')

In [77]:
charge_order = pd.concat([charge_order_12,charge_order_15])
charge_order

,日期,充电时段,经纬度编号,充电次数,总充电量,BEV车辆数,PHEV车辆数,lon,lat,index
0,2023-07-12,0-1,121.46-31.34,100,"3,046.6",96,4,121.46,31.34,0
1,2023-07-12,22-23,121.46-31.34,69,"1,932.46",59,9,121.46,31.34,0
2,2023-07-12,5-6,121.46-31.34,66,"2,028.03",63,1,121.46,31.34,0
3,2023-07-12,14-15,121.46-31.34,63,"1,910.07",56,4,121.46,31.34,0
4,2023-07-12,11-12,121.46-31.34,32,637.77,27,3,121.46,31.34,0
...,...,...,...,...,...,...,...,...,...,...
4470,2023-07-15,22-23,121.36-31.34,1,9.61,0,1,121.36,31.34,659
4471,2023-07-15,8-9,121.36-31.34,1,0.74,0,1,121.36,31.34,659
4472,2023-07-15,8-9,121.37-31.29,1,0.56,0,1,121.37,31.29,616
4473,2023-07-15,8-9,121.48-31.36,1,15.2,0,1,121.48,31.36,648


In [78]:
charge_order['time'] = pd.to_datetime(charge_order['日期'], format='%Y-%m-%d')
charge_order['day'] = charge_order['time'].dt.day
charge_order['hour'] = charge_order['充电时段'].apply(lambda x: int(x.split('-')[0]))
charge_order

,日期,充电时段,经纬度编号,充电次数,总充电量,BEV车辆数,PHEV车辆数,lon,lat,index,time,day,hour
0,2023-07-12,0-1,121.46-31.34,100,"3,046.6",96,4,121.46,31.34,0,2023-07-12,12,0
1,2023-07-12,22-23,121.46-31.34,69,"1,932.46",59,9,121.46,31.34,0,2023-07-12,12,22
2,2023-07-12,5-6,121.46-31.34,66,"2,028.03",63,1,121.46,31.34,0,2023-07-12,12,5
3,2023-07-12,14-15,121.46-31.34,63,"1,910.07",56,4,121.46,31.34,0,2023-07-12,12,14
4,2023-07-12,11-12,121.46-31.34,32,637.77,27,3,121.46,31.34,0,2023-07-12,12,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4470,2023-07-15,22-23,121.36-31.34,1,9.61,0,1,121.36,31.34,659,2023-07-15,15,22
4471,2023-07-15,8-9,121.36-31.34,1,0.74,0,1,121.36,31.34,659,2023-07-15,15,8
4472,2023-07-15,8-9,121.37-31.29,1,0.56,0,1,121.37,31.29,616,2023-07-15,15,8
4473,2023-07-15,8-9,121.48-31.36,1,15.2,0,1,121.48,31.36,648,2023-07-15,15,8


In [79]:
charge_order = charge_order[['day','hour','充电次数','index']]

In [80]:
charge_order

,day,hour,充电次数,index
0,12,0,100,0
1,12,22,69,0
2,12,5,66,0
3,12,14,63,0
4,12,11,32,0
...,...,...,...,...
4470,15,22,1,659
4471,15,8,1,659
4472,15,8,1,616
4473,15,8,1,648


In [81]:
train_data = pd.merge(charge_order,charge_agg,on=['index'],how='left')
train_data

,day,hour,充电次数,index,geometry,LONCOL,LATCOL,lon,lat,area,...,09城市绿地公共停车场,07交通枢纽公共停车场,06工业园区内部停车场,其它,11路边停车位,01公共,05其他专用,06虚拟桩群,02公交车专用,03出租车专用
0,12,0,100,0,"POLYGON ((121.45500 31.33500, 121.46500 31.335...",146.0,134.0,121.46,31.34,3.141321e+06,...,0.0,0.0,0.0,0.0,0.0,2.0,1.0,12.0,1.0,0.0
1,12,22,69,0,"POLYGON ((121.45500 31.33500, 121.46500 31.335...",146.0,134.0,121.46,31.34,3.141321e+06,...,0.0,0.0,0.0,0.0,0.0,2.0,1.0,12.0,1.0,0.0
2,12,5,66,0,"POLYGON ((121.45500 31.33500, 121.46500 31.335...",146.0,134.0,121.46,31.34,3.141321e+06,...,0.0,0.0,0.0,0.0,0.0,2.0,1.0,12.0,1.0,0.0
3,12,14,63,0,"POLYGON ((121.45500 31.33500, 121.46500 31.335...",146.0,134.0,121.46,31.34,3.141321e+06,...,0.0,0.0,0.0,0.0,0.0,2.0,1.0,12.0,1.0,0.0
4,12,11,32,0,"POLYGON ((121.45500 31.33500, 121.46500 31.335...",146.0,134.0,121.46,31.34,3.141321e+06,...,0.0,0.0,0.0,0.0,0.0,2.0,1.0,12.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9040,15,22,1,659,"POLYGON ((121.35500 31.33500, 121.36500 31.335...",136.0,134.0,121.36,31.34,7.661351e+05,...,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0
9041,15,8,1,659,"POLYGON ((121.35500 31.33500, 121.36500 31.335...",136.0,134.0,121.36,31.34,7.661351e+05,...,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0
9042,15,8,1,616,"POLYGON ((121.36500 31.28500, 121.37500 31.285...",137.0,129.0,121.37,31.29,5.417293e+05,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
9043,15,8,1,648,"POLYGON ((121.47500 31.35500, 121.48500 31.355...",148.0,136.0,121.48,31.36,5.205359e+06,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [82]:
train_data.columns

Index(['day', 'hour', '充电次数', 'index', 'geometry', 'LONCOL', 'LATCOL', 'lon',
       'lat', 'area', 'technology_park', 'car_park', 'supermarket',
       'residential', 'commercial', '交流桩数量', '直流桩数量', '交流桩总装机功率', '直流桩总装机功率',
       '01城市内道路', '02城际间高速道路', '道路unknown', '02居民区内部停车场', '01充电场站',
       '10大型建筑物配建停车场', '03公共机构内部停车场', '05写字楼内部停车场', '04企事业单位内部停车场',
       '08大型文体设施公共停车场', '09城市绿地公共停车场', '07交通枢纽公共停车场', '06工业园区内部停车场', '其它',
       '11路边停车位', '01公共', '05其他专用', '06虚拟桩群', '02公交车专用', '03出租车专用'],
      dtype='object')

In [83]:
train_data.drop(columns=['LONCOL', 'LATCOL'],inplace=True)

In [84]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9045 entries, 0 to 9044
Data columns (total 37 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   day              9045 non-null   int32   
 1   hour             9045 non-null   int64   
 2   充电次数             9045 non-null   int64   
 3   index            9045 non-null   int64   
 4   geometry         8793 non-null   geometry
 5   lon              8793 non-null   float64 
 6   lat              8793 non-null   float64 
 7   area             8793 non-null   float64 
 8   technology_park  8793 non-null   float64 
 9   car_park         8793 non-null   float64 
 10  supermarket      8793 non-null   float64 
 11  residential      8793 non-null   float64 
 12  commercial       8793 non-null   float64 
 13  交流桩数量            8793 non-null   float64 
 14  直流桩数量            8793 non-null   float64 
 15  交流桩总装机功率         8793 non-null   float64 
 16  直流桩总装机功率         8793 non-null   float64 


In [85]:
train_data.to_csv('./output/grid_train.csv',index=None)